Based on <https://cell2location.readthedocs.io/en/latest/notebooks/cell2location_short_demo.html>

In [ ]:
import os
import re

import IPython

import pandas as pd

import scanpy as sc

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# silence scanpy that prints a lot of warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Test configuration
os.chdir("/ceph/project/tendonhca/albrecht/003-snakemake/")
sample_name = "OMB1277_SSP_Enth_H"
genes_gtf = "transcriptome/refdata-gex-GRCh38-2020-A/genes/genes.gtf"

In [ ]:
genes_dict = {}
with open(genes_gtf, "rt") as stream_in:
    for line in stream_in:
        if line.startswith("#"):
            continue
        else:
            line_data = line.strip().split("\t")
            if line_data[2] == "gene":
                gene_name = re.search(r'gene_name \"(.+?)\";', line_data[8]).group(1)
                gene_id = re.search(r'gene_id \"(.+?)\";', line_data[8]).group(1)
                genes_dict[gene_id] = {"gene_name": gene_name, "seqname": line_data[0]}
genes_df = pd.DataFrame.from_dict(genes_dict, orient="index")
genes_df.head()

In [ ]:
gene_symbols_mt = genes_df[genes_df["seqname"] == "chrM"]["gene_name"].tolist()
len(gene_symbols_mt)

In [ ]:
gene_symbols_ribosomal = [gene_symbol for gene_symbol in genes_df["gene_name"] if gene_symbol.startswith(("RPL", "RPS")) ]
len(gene_symbols_ribosomal)

In [ ]:
def read_and_qc(sample_name):
    r""" This function reads the data for one 10X spatial experiment into the anndata object.
    It also calculates QC metrics. Modify this function if required by your workflow.
    
    :param sample_name: Name of the sample
    """
    
    adata = sc.read_visium("results/spaceranger_count/" + str(sample_name) + '/outs',
                           count_file='filtered_feature_bc_matrix.h5', load_images=True)
    adata.obs['sample'] = sample_name
    adata.var['SYMBOL'] = adata.var_names
    adata.var.rename(columns={'gene_ids': 'ENSEMBL'}, inplace=True)
    adata.var_names = adata.var['ENSEMBL']
    adata.var.drop(columns='ENSEMBL', inplace=True)
    
    # Calculate QC metrics
    from scipy.sparse import csr_matrix
    adata.X = adata.X.toarray()
    sc.pp.calculate_qc_metrics(adata, inplace=True)
    adata.X = csr_matrix(adata.X)
    adata.var['mt'] = [gene in gene_symbols_mt for gene in adata.var['SYMBOL']]
    adata.obs['mt_frac'] = adata[:, adata.var['mt'].tolist()].X.sum(1).A.squeeze()/adata.obs['total_counts']
    adata.var['ribosomal'] = [gene in gene_symbols_ribosomal for gene in adata.var['SYMBOL']]
    adata.obs['ribosomal_frac'] = adata[:, adata.var['ribosomal'].tolist()].X.sum(1).A.squeeze()/adata.obs['total_counts']
    
    # add sample name to obs names
    adata.obs["sample"] = [str(i) for i in adata.obs['sample']]
    adata.obs_names = adata.obs["sample"] \
                          + '_' + adata.obs_names
    adata.obs.index.name = 'spot_id'
    
    return adata

def select_slide(adata, s, s_col='sample'):
    r""" This function selects the data for one slide from the spatial anndata object.

    :param adata: Anndata object with multiple spatial experiments
    :param s: name of selected experiment
    :param s_col: column in adata.obs listing experiment name for each location
    """
    
    slide = adata[adata.obs[s_col].isin([s]), :]
    s_keys = list(slide.uns['spatial'].keys())
    s_spatial = np.array(s_keys)[[s in k for k in s_keys]][0]
    
    slide.uns['spatial'] = {s_spatial: slide.uns['spatial'][s_spatial]}
    
    return slide

In [ ]:
# Read the data into anndata objects#
adata = read_and_qc(sample_name)
adata

adata.lobs

In [ ]:
# mitochondria-encoded (MT) genes should be removed for spatial mapping
adata.obsm['mt'] = adata[:, adata.var['mt'].values].X.toarray()
adata = adata[:, ~adata.var['mt'].values]
adata

In [ ]:
# remove ribosomal genes
adata.obsm['ribosomal'] = adata[:, adata.var['ribosomal'].values].X.toarray()
adata = adata[:, ~adata.var['ribosomal'].values]
adata

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(15, 6))
slide = select_slide(adata, sample_name)
sns.distplot(slide.obs['total_counts'],
                kde=False, ax = axs[0, 0])
axs[0, 0].set_xlim(0, adata.obs['total_counts'].max())
axs[0, 0].set_xlabel(f'total_counts | {sample_name}')
x_max = np.quantile(slide.obs['total_counts'], .9)
sns.distplot(slide.obs['total_counts']\
                [slide.obs['total_counts']<x_max],
                kde=False, bins=40, ax = axs[0, 1])
axs[0, 1].set_xlim(0, x_max)
axs[0, 1].set_xlabel(f'total_counts | {sample_name}')

sns.distplot(slide.obs['n_genes_by_counts'],
                kde=False, bins=60, ax = axs[0, 2])
axs[0, 2].set_xlim(0, adata.obs['n_genes_by_counts'].max())
axs[0, 2].set_xlabel(f'n_genes_by_counts | {sample_name}')
x_max = np.quantile(slide.obs['n_genes_by_counts'], .9)
sns.distplot(slide.obs['n_genes_by_counts']\
                [slide.obs['n_genes_by_counts']<x_max],
                kde=False, bins=60, ax = axs[0, 3])
axs[0, 3].set_xlim(0, x_max)
axs[0, 3].set_xlabel(f'n_genes_by_counts | {sample_name}')

sns.distplot(slide.obs['mt_frac'],
                kde=False, bins=60, ax = axs[1, 0])
axs[1, 0].set_xlim(0, adata.obs['mt_frac'].max())
axs[1, 0].set_xlabel(f'mt_frac | {sample_name}')
x_max = np.quantile(slide.obs['mt_frac'], .9)
sns.distplot(slide.obs['mt_frac']\
                [slide.obs['mt_frac']<x_max],
                kde=False, bins=60, ax = axs[1, 1])
axs[1, 1].set_xlim(0, x_max)
axs[1, 1].set_xlabel(f'mt_frac | {sample_name}')

sns.distplot(slide.obs['ribosomal_frac'],
                kde=False, bins=60, ax = axs[1, 2])
axs[1, 2].set_xlim(0, adata.obs['ribosomal_frac'].max())
axs[1, 2].set_xlabel(f'ribosomal_frac | {sample_name}')
x_max = np.quantile(slide.obs['ribosomal_frac'], .9)
sns.distplot(slide.obs['ribosomal_frac']\
                [slide.obs['ribosomal_frac']<x_max],
                kde=False, bins=60, ax = axs[1, 3])
axs[1, 3].set_xlim(0, x_max)
axs[1, 3].set_xlabel(f'ribosomal_frac | {sample_name}')
    
plt.tight_layout()

In [ ]:
slide = select_slide(adata, sample_name)

with mpl.rc_context({'figure.figsize': [6,7],
                     'axes.facecolor': 'white'}):
    sc.pl.spatial(slide, img_key = "hires", cmap='magma', ncols = 3,
                  library_id=sample_name,
                  color=['log1p_total_counts', 'total_counts', 'n_genes_by_counts', 'mt_frac', 'ribosomal_frac'], size=1,
                  vmin = 0, vmax='p90.0',
                  gene_symbols='SYMBOL', show=False, return_fig=True)

In [ ]:
slide = select_slide(adata, sample_name)

with mpl.rc_context({'figure.figsize': [6,7],
                     'axes.facecolor': 'white'}):
    sc.pl.spatial(slide, img_key = "hires", cmap='magma',
                  library_id=sample_name,
                  color=['total_counts', 'n_genes_by_counts', 'mt_frac', 'ribosomal_frac'], size=1, vmax='p90.0',
                  gene_symbols='SYMBOL', show=False, return_fig=True)

In [ ]:
IPython.display.Image(filename='images/omb1277_ssp_enth_h.jpeg', width=500)

In [ ]:
adata_normalized = adata.copy()
sc.pp.normalize_total(adata_normalized)
adata_normalized_X_feature_mean = np.array(adata_normalized.X.mean(axis=0)).flatten()
d = {'SYMBOL': adata_normalized.var['SYMBOL'], 'mean': adata_normalized_X_feature_mean}
adata_normalized_X_feature_mean = pd.DataFrame(d, index=adata_normalized.var_names).sort_values(by=['mean'], ascending=False)
adata_normalized_X_feature_mean.head()

In [ ]:
with mpl.rc_context({'figure.figsize': [6,7],
                     'axes.facecolor': 'black'}):
    sc.pl.spatial(adata_normalized,
                  color=adata_normalized_X_feature_mean.head(12)['SYMBOL'], img_key=None, size=1,
                  vmin=0, cmap='magma', vmax='p90.0',
                  gene_symbols='SYMBOL'
                 )


In [ ]:
markers_df = pd.read_csv('data/curated_markers.tsv', sep="\t")
markers_df.head()

In [ ]:
cell_type = 'Fibroblast'
markers_symbols = markers_df[markers_df['cell_type'] == cell_type]['gene_symbol']
markers_symbols = set(markers_symbols).intersection(adata_normalized.var['SYMBOL'])
with mpl.rc_context({'figure.figsize': [6,7],
                     'axes.facecolor': 'black'}):
    sc.pl.spatial(adata_normalized,
                  color=markers_symbols, img_key=None, size=1,
                  vmin=0, cmap='magma', vmax='p90.0',
                  gene_symbols='SYMBOL'
                 )

In [ ]:
cell_type = 'Adipocyte'
markers_symbols = markers_df[markers_df['cell_type'] == cell_type]['gene_symbol']
markers_symbols = set(markers_symbols).intersection(adata_normalized.var['SYMBOL'])
with mpl.rc_context({'figure.figsize': [6,7],
                     'axes.facecolor': 'black'}):
    sc.pl.spatial(adata_normalized,
                  color=markers_symbols, img_key=None, size=1,
                  vmin=0, cmap='magma', vmax='p90.0',
                  gene_symbols='SYMBOL'
                 )

In [ ]:
cell_type = 'Skeletal muscle'
markers_symbols = markers_df[markers_df['cell_type'] == cell_type]['gene_symbol']
markers_symbols = set(markers_symbols).intersection(adata_normalized.var['SYMBOL'])
with mpl.rc_context({'figure.figsize': [6,7],
                     'axes.facecolor': 'black'}):
    sc.pl.spatial(adata_normalized,
                  color=markers_symbols, img_key=None, size=1,
                  vmin=0, cmap='magma', vmax='p90.0',
                  gene_symbols='SYMBOL'
                 )

In [ ]:
cell_type = 'Vascular endothelium'
markers_symbols = markers_df[markers_df['cell_type'] == cell_type]['gene_symbol']
markers_symbols = set(markers_symbols).intersection(adata_normalized.var['SYMBOL'])
with mpl.rc_context({'figure.figsize': [6,7],
                     'axes.facecolor': 'black'}):
    sc.pl.spatial(adata_normalized,
                  color=markers_symbols, img_key=None, size=1,
                  vmin=0, cmap='magma', vmax='p90.0',
                  gene_symbols='SYMBOL'
                 )

## Construct and examine UMAP of locations

As per <https://cell2location.readthedocs.io/en/latest/notebooks/cell2location_short_demo.html>

In [ ]:
adata_vis = adata.copy()
adata_vis.raw = adata_vis

adata_vis_plt = adata_vis.copy()

# NOTE (Kevin): no normalisation to total count per spot?!
sc.pp.normalize_total(adata_vis_plt, target_sum=1e4)

# Log-transform (log(data + 1))
sc.pp.log1p(adata_vis_plt)

# Find highly variable genes within each sample
adata_vis_plt.var['highly_variable'] = False
sc.pp.highly_variable_genes(adata_vis_plt, min_mean=0.05, max_mean=5, min_disp=1)

hvg_list = list(adata_vis_plt.var_names[adata_vis_plt.var['highly_variable']])
adata_vis_plt.var.loc[hvg_list, 'highly_variable'] = True

sc.pl.highly_variable_genes(adata_vis_plt)

In [ ]:
len(hvg_list)

In [ ]:
# Scale the data ( (data - mean) / sd )
sc.pp.scale(adata_vis_plt, max_value=10)
# PCA, KNN construction, UMAP
sc.tl.pca(adata_vis_plt, svd_solver='arpack', n_comps=50, use_highly_variable=True)
# ... continued below ...

QC of PCA prior to choosing number of components

In [ ]:
plot_data = pd.DataFrame({
    'PC': range(len(adata_vis_plt.uns['pca']['variance_ratio'])),
    'variance_ratio': adata_vis_plt.uns['pca']['variance_ratio'],
    })
plot_data.head()
plot_data.plot.scatter(x='PC', y='variance_ratio')

In [ ]:
sc.pp.neighbors(adata_vis_plt, n_neighbors = 10, n_pcs = 10, metric='cosine')
sc.tl.umap(adata_vis_plt, min_dist = 0.3, spread = 1)

with mpl.rc_context({'figure.figsize': [8, 8],
                     'axes.facecolor': 'white'}):
    sc.pl.umap(adata_vis_plt, color=['sample'], size=30,
               color_map = 'RdPu', ncols = 1, #legend_loc='on data',
               legend_fontsize=10)